In [1]:
import pandas as pd
import numpy as np
import warnings
import gspread

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from datetime import datetime, timedelta
from google.colab import drive, auth
auth.authenticate_user()
from google.auth import default
creds, _= default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe


In [2]:
#function to generate synthetic traffic, cta and conversion data
def engineer_traffic_data(start_date, end_date):
    dates = pd.date_range(start_date, end_date)

    data = []

    #defining medium, source, adset and campaign values
    mediums = ['paid social', 'organic social', 'paid search', 'paid review', 'direct']
    sources = {'paid social': ['facebook', 'twitter', 'linkedin'], 'organic social': ['facebook', 'twitter', 'linkedin'], 'paid search': ['google', 'bing'], 'paid review': ['capterra', 'g2'], 'direct': ['direct']}

    campaigns = {'facebook': ['facebook_awareness', 'facebook_conversion'], 'twitter': ['twitter_traffic', 'twitter_engagement'], 'linkedin': ['linkedin_leads', 'linkedin_conversion'], 'google': ['google_search', 'google_display'], 'bing': ['bing_search'], 'capterra': ['capterra_review'], 'g2': ['g2_review'], 'direct': ['direct']}

    adsets = {'facebook_awareness': ['facebook_img1', 'facebook_video1'], 'facebook_conversion': ['facebook_img2', 'facebook_video2'], 'twitter_traffic': ['twitter_img1', 'twitter_video1'], 'twitter_engagement': ['twitter_img2', 'twitter_video2'], 'linkedin_leads': ['linkedin_img1', 'linkedin_video1'], 'linkedin_conversion': ['linkedin_img2', 'linkedin_video2'], 'google_search': ['google_ad1', 'google_ad2'], 'google_display': ['google_display1', 'google_display2'], 'bing_search': ['bing_ad1', 'bing_ad2'], 'capterra_review': ['capterra_ad1'], 'g2_review': ['g2_ad1'], 'direct': ['direct']}

    #iterating through to create values
    for date in dates:
        for medium in mediums:
            for source in sources[medium]:
                for campaign in campaigns[source]:
                    for adset in adsets[campaign]:
                        unique_visitors = np.random.randint(0, 51)
                        if unique_visitors == 0:
                            continue
                        new_visitors = np.random.randint(0, unique_visitors + 1)
                        pricing_page_visited = np.random.randint(0, new_visitors + 1)
                        demo_video_played = np.random.randint(0, pricing_page_visited + 1)
                        testimonial_cta = np.random.randint(0, pricing_page_visited + 1)
                        engagement_rate = np.random.uniform(0, 0.62)
                        duration = timedelta(minutes=np.random.uniform(2, 5))
                        wp_selected = np.random.randint(0, pricing_page_visited + 1)
                        wp_downloaded = np.random.randint(0, wp_selected + 1)
                        demo_cta = np.random.randint(0, pricing_page_visited + 1)
                        demo_booked = np.random.randint(0, demo_cta + 1)
                        mql = np.random.randint(0, demo_booked + 1)
                        demo_attended = np.random.randint(0, mql + 1)
                        sql = np.random.randint(0, demo_attended + 1)
                        opportunities = np.random.randint(0, sql + 1)
                        converted = np.random.randint(0, opportunities + 1)
                        deal = np.random.randint(10000, 150001) if converted > 0 else 0
                        spend = 0
                        if medium in ['paid social', 'paid search', 'paid review']:
                            spend = np.random.randint(5, 16)

                        data.append({
                            'received_at': date,
                            'medium': medium,
                            'source': source,
                            'campaign': campaign,
                            'adset': adset,
                            'unique_visitors': unique_visitors,
                            'new_visitors': new_visitors,
                            'pricing_page_visited': pricing_page_visited,
                            'demo_video_played': demo_video_played,
                            'engagement_rate': engagement_rate,
                            'duration': duration,
                            'testimonial_cta': testimonial_cta,
                            'wp_selected': wp_selected,
                            'wp_downloaded': wp_downloaded,
                            'demo_cta': demo_cta,
                            'demo_booked': demo_booked,
                            'mql': mql,
                            'demo_attended': demo_attended,
                            'sql': sql,
                            'opportunities': opportunities,
                            'converted': converted,
                            'deal': deal,
                            'spend': spend
                            })

    return pd.DataFrame(data)

#defining start date, end date and engineering data
start_date = datetime.now() - timedelta(days=180)
end_date = datetime.now()
traffic_data = engineer_traffic_data(start_date, end_date)
traffic_data.head()


,received_at,medium,source,campaign,adset,unique_visitors,new_visitors,pricing_page_visited,demo_video_played,engagement_rate,duration,testimonial_cta,wp_selected,wp_downloaded,demo_cta,demo_booked,mql,demo_attended,sql,opportunities,converted,deal,spend
0,2023-12-20 23:48:41.830001,paid social,facebook,facebook_awareness,facebook_img1,26,20,3,2,0.438650,0 days 00:03:00.836160,1,1,0,2,0,0,0,0,0,0,0,11
1,2023-12-20 23:48:41.830001,paid social,facebook,facebook_awareness,facebook_video1,11,11,5,3,0.346486,0 days 00:02:29.510996,5,5,2,1,1,1,1,1,1,0,0,5
2,2023-12-20 23:48:41.830001,paid social,facebook,facebook_conversion,facebook_img2,41,14,6,6,0.178333,0 days 00:02:05.923753,0,5,5,0,0,0,0,0,0,0,0,14
3,2023-12-20 23:48:41.830001,paid social,facebook,facebook_conversion,facebook_video2,12,5,5,3,0.208429,0 days 00:03:51.078342,5,2,0,4,3,1,1,0,0,0,0,9
4,2023-12-20 23:48:41.830001,paid social,twitter,twitter_traffic,twitter_img1,38,0,0,0,0.505497,0 days 00:04:30.156305,0,0,0,0,0,0,0,0,0,0,0,7


In [3]:
worksheet = gc.open('Portfolio_data')
worksheet = worksheet.worksheet('all traffic with cta')
worksheet.clear()
set_with_dataframe(worksheet, traffic_data)

[link to dashboard](https://lookerstudio.google.com/reporting/b7f7aeae-c645-4a80-9b3a-03b01b95c7f6)